In [1]:
# Importing libraries
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer

import re

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

C:\Users\Sai charan\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")
C:\Users\Sai charan\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
con1 = sqlite3.connect('final.sqlite')
filtered_data = pd.read_sql_query(" SELECT * FROM Reviews ", con1)


print(filtered_data.shape)
filtered_data.head()

(364171, 12)


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak book watch realli rosi movi i...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn month year learn poem t...
3,138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great littl book read nice rhythm well good re...
4,138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetri month year goe month cute littl po...


# Data Cleaning: Deduplication

In [3]:
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
print(final.shape)

#Checking to see how much % of data still remains
((final.shape[0]*1.0)/(filtered_data.shape[0]*1.0)*100)

(364171, 12)


100.0

In [4]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

print(final.shape)
final[30:50]

(364171, 12)


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
12,138700,150518,0006641040,AK1L4EJBA23JF,L. M. Kraus,0,0,positive,1288224000,love this book,"Great book, perfect condition arrived in a sho...",great book perfect condit arriv short amount t...
13,138701,150519,0006641040,A12HY5OZ2QNK4N,Elizabeth H. Roessner,0,0,positive,1256774400,It's a great book!,I've always loved chicken soup and rice. My la...,ive alway love chicken soup rice late ethel al...
14,138702,150520,0006641040,ADBFSA9KTQANE,"James L. Hammock ""Pucks Buddy""",0,0,positive,1256688000,Great Gift,This book was purchased as a birthday gift for...,book purchas birthday gift year old boy squeal...
8,138696,150514,0006641040,A2ONB6ZA292PA,Rosalind Matzner,0,0,positive,1313884800,Chicken soup with Rice,It's a great book with adorable illustrations....,great book ador illustr true classic kid love ...
34,476617,515426,141278509X,AB1A5EGHHVA9M,CHelmic,1,1,positive,1332547200,The best drink mix,This product by Archer Farms is the best drink...,product archer farm best drink mix ever mix fl...
36,22620,24750,2734888454,A13ISQV0U9GZIC,Sandikaye,1,1,negative,1192060800,made in china,My dogs loves this chicken but its a product f...,dog love chicken product china wont buy anymor...
35,22621,24751,2734888454,A1C298ITT645B6,Hugh G. Pritchard,0,0,positive,1195948800,Dog Lover Delites,Our dogs just love them. I saw them in a pet ...,dog love saw pet store tag attach regard made ...
37,284375,308077,2841233731,A3QD68O22M2XHQ,LABRNTH,0,0,positive,1345852800,Great recipe book for my babycook,This book is easy to read and the ingredients ...,book easi read ingredi avail store unlik recip...
142,157910,171225,7310172001,A314APAWYQFKBJ,"Diana Hersholt ""dog lover""",1,1,positive,1104796800,My dog loves these!,My dog is a very picky eater. She has a very ...,dog picki eater sensit stomach special food tr...
143,157909,171224,7310172001,AK0CENM3LUM28,Ana Mardoll,1,1,positive,1222732800,Pure Puppy Addiction,Pro-Treat Freeze Dried Liver Dog Treats / B000...,freez dri liver dog treat sort product make wi...


# Stemming, stop-word removal and Lemmatization

In [5]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
words_to_keep = set(('not'))
stop -= words_to_keep
#initialising the snowball stemmer
sno = nltk.stem.SnowballStemmer('english')

 #function to clean the word of any html-tags
def cleanhtml(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

#function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [6]:
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    
    
    final_string.append(str1)
    i+=1

In [7]:
final['CleanedText']=final_string  
final['CleanedText']=final['CleanedText'].str.decode("utf-8")
#below the processed review can be seen in the CleanedText Column 
print('Shape of final',final.shape)
final.head()

Shape of final (364171, 12)


,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
19,138708,150526,0006641040,A3E9QZFE9KXH8J,R. Mitchell,11,18,negative,1129507200,awesome book poor size,This is one of the best children's books ever ...,one best children book ever written mini versi...
20,138709,150529,0006641040,A25ACLV5KPB4W,"Matt Hetling ""Matt""",0,1,positive,1108425600,"Nice cadence, catchy rhymes",In June<br />I saw a charming group<br />of ro...,june saw charm group rose begin droop pep chic...
21,138699,150517,0006641040,ABW4IC5G5G8B5,kevin clark,0,0,positive,1291075200,good for children,"Classic children's book, can't go wrong. I rea...",classic children book cant wrong read kid orde...
22,138686,150504,0006641040,AQEYF1AXARWJZ,"Les Sinclair ""book maven""",1,1,positive,1212278400,Chicken Soup with Rice,A very entertaining rhyming story--cleaver and...,entertain rhyme catchi illustr imagin fit righ...


In [8]:
##Sorting data according to Time in ascending order for Time Based Splitting
time_sorted_data = final.sort_values('Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

# We will collect different 5K rows without repetition from time_sorted_data dataframe
my_final = time_sorted_data.take(np.random.permutation(len(final))[:5000])

x = my_final['CleanedText'].values

# Bag of Words (BoW)

In [9]:
count_vect = CountVectorizer(min_df = 10) 
data = count_vect.fit_transform(x)
print("the type of count vectorizer :",type(data))
print("the shape of out text BOW vectorizer : ",data.get_shape())
print("the number of unique words :", data.get_shape()[1])

the type of count vectorizer : <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer :  (5000, 1909)
the number of unique words : 1909


# Hierarchical Clustering with 2 clusters

In [10]:
from sklearn.cluster import AgglomerativeClustering

model = AgglomerativeClustering(n_clusters=2).fit(data.toarray())

reviews = my_final['Text'].values
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    else :
        cluster2.append(reviews[i])
 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))

No. of reviews in Cluster-1 :  243

No. of reviews in Cluster-2 :  4757


In [19]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

READING REVIEWS MANUALLY

In [15]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 I love holiday candies because they're predictably fresh (as opposed to the regular versions that have been sitting in a warehouse for 6 months - the holiday packaging ensures they're new stock).  Bought these from amazon for a great price despite mediocre reviews.  The reviews are true - this stuff wasn't handled with care - but the candies are still terrific.  They're visually a little waxy (visual eaters take note!) but the flavor is just a notch or two below ideal freshness and at least two notches above the non-holiday versions typically on my grocer's shelf.  Quite good.<br /><br />If you typically eat the Peanut Butter Cup (PBC) form, you may find the flavor of the holiday versions a little different.  The different shapes of the various holiday versions (bunnies, eggs, Christmas trees, etc.) alter the balance of peanut butter to chocolate in each bite.  There is a small finch-egg size PB egg (PBE) that is harder like the mini-cups.  It's actually interesting to gee

In [12]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-3 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enj

In [ ]:
print("Cluster 2 : \n")
count=1
for review in cluster2[0:3]:
    print("Review {0} : {1}".format(count,review))
    # Plot word cloud of review
    plot_word_cloud(review)
    print("\n\n\n")
    count +=1

# Hierarchical Clustering with 5 clusters

In [13]:
model = AgglomerativeClustering(n_clusters=5).fit(data.toarray())

# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    else :
        cluster5.append(reviews[i]) 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5))

No. of reviews in Cluster-1 :  543

No. of reviews in Cluster-2 :  243

No. of reviews in Cluster-3 :  1

No. of reviews in Cluster-4 :  3943

No. of reviews in Cluster-5 :  270


In [14]:
 #Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 It's very difficult to tell if this product is working. I applied it around my deck to repel rabbits. We have them in the yard, and sometimes they hover around the hostas around the deck. I haven't noticed any rabbits near the house since application, but we had rain nearly every day for a week, so I'm not sure if the product is effective anymore or not. I also sprinkled it around one of my gardens that doesn't have a fence to deter rabbits, groundhogs and moles. Since I don't have a camera set up, and I'm not monitoring it round the clock, I just can't tell if it's keeping all or some of them away.<br /><br />When I sprinkle it on, the product has a very strong smell of black pepper. It's easy to apply, but doesn't go very far. It's a little expensive for me to purchase for additional applications.<br /><br />For those wondering what the other 99% of inactive ingredients might be, I'm 99% sure it's sand. So, this is black pepper, Piperine, and Capsaicin, and a ton of sand

In [15]:
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 This English Breakfast tea from Higgins & Burke, is good: it's basic, good tasting black tea (English Breakfast blend) and is tasty.  I wasn't blown away by the flavor, but it was good: I enjoyed the tea, it brewed up quickly in these nice tea bags (I recommend about one minute of brew time), and then added a splash of cream: it was quite nice!

Review-2 : 
 I have never had black tea this good before.  I love lychee fruit and wanted some black tea for the caffeine content.  Problem is I do not like the taste of black tea.  This tea is different it is not bitter.  It is smooth in taste.  I do not add sugar to my teas so they have to stand on their own.  This one does.  That is why I purchase teas with flavors in them.  This may seem expensive, but realize this is 100 bags of tea and comes in a huge box.  If you look a regular tea it is a much smaller box.  This looks over three times the size.  Great tea try it for your caffenated tea.  It is wonderful tea.  This beats out

In [16]:
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 QUALITY<br />In my opinion, having used a dozen or so brands of olive oil, Colavita is simply the best reasonably priced olive oil which is widely available.  I've been using Colavita for several years now, and have never been dissappointed.  Colavita is excellent quality, dependable, and reasonably priced---the perfect combination for a daily general purpose olive oil.<br /><br />FLAVOR<br />Although not an "olive oil gourmand", I'd describe Colavita as "rich, smooth, and neutral". Gourmands probably have other silly terms for the flavor, such as "fruity" (but a "civilian" would never call it fruity). It is not harsh or bitter like most common olive oil, not spicy like Whole Foods 360, and not notably "olivey".  Colavita is an evoo ("extra virgin olive oil"---to us non-olive oil snobs), which means "first pressing".  Most olive-oil nuts consider the words "extra virgin" to be sacred.  Practically speaking though, the mere label "extra virgin" is not a guarantee of quality

In [17]:
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-3 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enj

In [ ]:
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

# Hierarchical Clustering with 10 clusters

In [18]:
model = AgglomerativeClustering(n_clusters=10).fit(data.toarray())
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []
cluster6 = []
cluster7 = []
cluster8 = []
cluster9 = []
cluster10 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    elif model.labels_[i] == 4:
        cluster5.append(reviews[i])
    elif model.labels_[i] == 5:
        cluster6.append(reviews[i])
    elif model.labels_[i] == 6:
        cluster7.append(reviews[i])
    elif model.labels_[i] == 7:
        cluster8.append(reviews[i])
    elif model.labels_[i] == 8:
        cluster9.append(reviews[i])       
    else :
        cluster10.append(reviews[i])

In [19]:
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5)) 
print("\nNo. of reviews in Cluster-6 : ",len(cluster6))
print("\nNo. of reviews in Cluster-7 : ",len(cluster7))
print("\nNo. of reviews in Cluster-8 : ",len(cluster8))
print("\nNo. of reviews in Cluster-9 : ",len(cluster9))
print("\nNo. of reviews in Cluster-10 : ",len(cluster10))

No. of reviews in Cluster-1 :  316

No. of reviews in Cluster-2 :  2

No. of reviews in Cluster-3 :  224

No. of reviews in Cluster-4 :  1117

No. of reviews in Cluster-5 :  270

No. of reviews in Cluster-6 :  1

No. of reviews in Cluster-7 :  1

No. of reviews in Cluster-8 :  199

No. of reviews in Cluster-9 :  44

No. of reviews in Cluster-10 :  2826


In [20]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 It's very difficult to tell if this product is working. I applied it around my deck to repel rabbits. We have them in the yard, and sometimes they hover around the hostas around the deck. I haven't noticed any rabbits near the house since application, but we had rain nearly every day for a week, so I'm not sure if the product is effective anymore or not. I also sprinkled it around one of my gardens that doesn't have a fence to deter rabbits, groundhogs and moles. Since I don't have a camera set up, and I'm not monitoring it round the clock, I just can't tell if it's keeping all or some of them away.<br /><br />When I sprinkle it on, the product has a very strong smell of black pepper. It's easy to apply, but doesn't go very far. It's a little expensive for me to purchase for additional applications.<br /><br />For those wondering what the other 99% of inactive ingredients might be, I'm 99% sure it's sand. So, this is black pepper, Piperine, and Capsaicin, and a ton of sand

In [21]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 "We use and believe in stone milling because no machinery has yet been developed that grinds grains into flours, cereals, and meals quite as well as the flint-hard quartz stones used by master millers since early Roman times." ~Bob's Red Mill Site<br /><br />The 100 % Stone Ground Whole Wheat Flour is my favorite flour for use in a variety of whole wheat bread recipes. It is all natural and doesn't contain any preservatives. You can use this flour for bread machine baking too and it produces superior results. I've used the recipe on the back of the bag and adapted it by using a mixture of whole wheat and regular bread flour because I didn't have any "gluten" as called for in the recipe. If you substitute 1 ½ cups of regular bread flour for part of the whole wheat flour you don't seem to need the gluten.<br /><br />Bob's Red Mill products remind me of my childhood when my father took me to the Graue Mill in Oakbrook, IL where we watched corn being milled. It is now the only

In [22]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 The dog food got good ratings at consumers, so I bought it and the dogs just love it.. It seems to be a very healtful and complete food.. I just add a little hot water and its gobbled up in no time

Review-2 : 
 In terms of convenience and taste, these are great. Overall, I have only one complaint. Given the design of the pouch it's pretty much impossible to get all the food out, which seems like a waste every time. It's not much, but more so than a regular glass container of baby food.<br /><br />Breakdown:<br /><br />PROS:<br />1. Baby loves it, and savors it.<br />2. Organic ingredients<br />3. Super easy to open (simple twist)<br />4. Tastes great (yeah, I try her food to make sure we're giving her food that has some flavor to it)<br />5. 4.5 ounces of food (larger than the normal/standard of 4 ounces.<br /><br />Neutral<br />1. In general I wish that baby food emphasized more the vegetables than the fruit. (these are plenty sweet, they could have easily used half as m

In [23]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enjoy.

Review-3 : 
 Kitchens of India is the closest to a homemade curry we have ever had. We love it. The sauce is great and it is fast, so curry a few times a week is possible. We often make a curry from scratch but it is time consuming and now we can have a good curry whenever we want.  We have also tried the Rogan Josh as well as the Korma. Add some onions and raisins, or a bit of honey. The ingredients have little or no

In [24]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-2 : 
 I bought this coffee on a whim as it was a little cheaper than green mountain.  I really like the flavor, it is a great medium flavor coffee.  I will purchase again.

Review-3 : 
 Marley Coffee & Tea One Love Organic Coffee makes a tasty cup of coffee.  However, I have to say that nowhere on the box, or the individual packets, are instructions for actually making the coffee.  Also it doesn't tell you what type of coffee maker to use or even how much water to use for each packet.  I use K-cups and my daughter uses T-cups.  I had to make this product in my old automatic drip coffee maker, and I had to guess on how much water to use.  Once I was able to make a cup, it was very good.



In [25]:
# Three Reviews of cluster 6
count=1
for i in range(3):
    if i < len(cluster6):
        print('Review-%d : \n %s\n'%(count,cluster6[i]))
        count +=1

Review-1 : 
 QUALITY<br />In my opinion, having used a dozen or so brands of olive oil, Colavita is simply the best reasonably priced olive oil which is widely available.  I've been using Colavita for several years now, and have never been dissappointed.  Colavita is excellent quality, dependable, and reasonably priced---the perfect combination for a daily general purpose olive oil.<br /><br />FLAVOR<br />Although not an "olive oil gourmand", I'd describe Colavita as "rich, smooth, and neutral". Gourmands probably have other silly terms for the flavor, such as "fruity" (but a "civilian" would never call it fruity). It is not harsh or bitter like most common olive oil, not spicy like Whole Foods 360, and not notably "olivey".  Colavita is an evoo ("extra virgin olive oil"---to us non-olive oil snobs), which means "first pressing".  Most olive-oil nuts consider the words "extra virgin" to be sacred.  Practically speaking though, the mere label "extra virgin" is not a guarantee of quality

In [26]:
# Three Reviews of cluster 7
count=1
for i in range(3):
    if i < len(cluster7):
        print('Review-%d : \n %s\n'%(count,cluster7[i]))
        count +=1

Review-1 : 
 Review:<br />I've eaten ramen for a long time (30+ years) and have eaten numerous varieties and the Shin ramen is probably my #1 tasting. The reasons are as follows:<br />* Best flavor broth - it's spicy (but adjustable), leaves a good aftertaste (yes, even when you burp), tint of mushroom flavor coupled with a good balance of red pepper, onions and spices.<br />* Noodles have good texture - depending how you cook them of course but with bad ones, no matter how you cook them they come out soggy/soft. These come out chewy if you want or softer if you overcook them.<br />* It's not oily - I've noticed with some noodles, you can see a ring of oil around the pot after you cook the noodles. So with these I sometimes had to boil an extra pot of water and then transfer the cooked noodles to the fresh pot of water to get rid of the oil. Not with Shin Ramen.<br />* Economical - for less than a dollar a pack, they beat any dollar menu item<br />* Great for any weather - particularly

In [27]:
# Three Reviews of cluster 8
count=1
for i in range(3):
    if i < len(cluster8):
        print('Review-%d : \n %s\n'%(count,cluster8[i]))
        count +=1

Review-1 : 
 This English Breakfast tea from Higgins & Burke, is good: it's basic, good tasting black tea (English Breakfast blend) and is tasty.  I wasn't blown away by the flavor, but it was good: I enjoyed the tea, it brewed up quickly in these nice tea bags (I recommend about one minute of brew time), and then added a splash of cream: it was quite nice!

Review-2 : 
 I purchased this tea because of all the good reviews.  I like it very much and will order again. Maybe Davidson's has change the ingredients because I found a lot of rosehips and had to really search for blueberries and found only 1. I also have the chamomile flowers and mixed that with the spring fruit and flowers in my tea pot, the combo made an excellent tea.  I do love rosehips and the flavor of the tea but was disappointed that I couldn't taste blueberry.  For this reason alone I gave this tea 4 stars.  I just got my second order of this tea and it was full of blueberries, no rosehips and has a blueberry taste to 

In [28]:
# Three Reviews of cluster 8
count=1
for i in range(3):
    if i < len(cluster9):
        print('Review-%d : \n %s\n'%(count,cluster9[i]))
        count +=1

Review-1 : 
 I have never had black tea this good before.  I love lychee fruit and wanted some black tea for the caffeine content.  Problem is I do not like the taste of black tea.  This tea is different it is not bitter.  It is smooth in taste.  I do not add sugar to my teas so they have to stand on their own.  This one does.  That is why I purchase teas with flavors in them.  This may seem expensive, but realize this is 100 bags of tea and comes in a huge box.  If you look a regular tea it is a much smaller box.  This looks over three times the size.  Great tea try it for your caffenated tea.  It is wonderful tea.  This beats out earl gray or english breakfast tea in taste.  This tea is a pleasure to drink.  I just wanted to get some caffeine in a tolerable way with no artificial sweeteners or regular sugars.  This is it.  In the highly sweetened american culture there is a way out drink this tea from china.  It is calorie free with no artifical sweeteners. I am not asian, but I love

In [ ]:
# Three Reviews of cluster 8
count=1
for i in range(3):
    if i < len(cluster10):
        print('Review-%d : \n %s\n'%(count,cluster10[i]))
        count +=1

# TFIDF

In [29]:
tf_idf_vect = TfidfVectorizer(min_df=10)
data = tf_idf_vect.fit_transform(x)
print("the type of count vectorizer :",type(data))
print("the shape of out text TFIDF vectorizer : ",data.get_shape())
print("the number of unique words :", data.get_shape()[1])

the type of count vectorizer : <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer :  (5000, 1931)
the number of unique words : 1931


# Hierarchical Clustering with 2 clusters

In [30]:
model = AgglomerativeClustering(n_clusters=2).fit(data.toarray())

reviews = my_final['Text'].values
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    else :
        cluster2.append(reviews[i])
 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))

No. of reviews in Cluster-1 :  4743

No. of reviews in Cluster-2 :  257


READING REVIEWS MANUALLY:

In [31]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    print('Review-%d : \n %s\n'%(count,cluster1[i]))
    count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-3 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enj

In [32]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    print('Review-%d : \n %s\n'%(count,cluster2[i]))
    count +=1

Review-1 : 
 Do not know a lot about this type of tea, so couldn't compare it to other brands. I can say the product is rather strong tasting, but not unpleasantly so. Since a portion is pretty small, that is not much of a problem anyway. I purchased it after reading about its many health benefits. It does open my eyes in the morning, so the caffeine in it is really nice. As recommended by others, I keep it in the refrigerator inside the tin, in its original package, and it's kept fresh-tasting for a couple of weeks now. The color is very bright green, and it does taste healthy--hopefully, it is.

Review-2 : 
 I have to say that this tea is very helpful during those stressful times.  I don't pay attention to the taste, but next thing I know, I'm like, "WOW - I'm so relaxed."  All that tension slowly melts away.  Great to have around during the time of the month.

Review-3 : 
 This English Breakfast tea from Higgins & Burke, is good: it's basic, good tasting black tea (English Breakfast

# Hierarchical Clustering with 5 clusters

In [33]:
model = AgglomerativeClustering(n_clusters=5).fit(data.toarray())

# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    else :
        cluster5.append(reviews[i]) 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5))

No. of reviews in Cluster-1 :  4019

No. of reviews in Cluster-2 :  262

No. of reviews in Cluster-3 :  358

No. of reviews in Cluster-4 :  257

No. of reviews in Cluster-5 :  104


READING REVIEWS MANUALLY:

In [34]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-3 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enj

In [35]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 I have a serious addiction to this gum but cannot find it anywhere except online. I go through at least a pack a day. I used to chew Peppermint or Spearmint or Winterfrost but I can't chew anything but Exotic Mint any more. The flavor is just right and it lasts forever.<br /><br />Seriously, if you see this gum in a store, buy all of it. It's like little candy gold bullion. I have opted to pay twice as much online for this gum versus gum I can buy on the store shelves just so I can get this particular flavor. Who would do that if the gum weren't the best gum ever? Nobody. That's who.<br /><br />I also wrote Wrigley to see if they'll send me a year-supply for free. Cross your fingers and wish me luck.

Review-2 : 
 We've had this for about a week now. I have a 1 year old Beagle that destroys ever furry "super strong" soft toy, but he isn't interested in a lot of hard toys like tires, etc. We got this hoping that his food motivation would keep him occupied with a toy he can'

In [36]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-2 : 
 Pilao is my favorite coffe. I have tried several brands from around the globe but so far this is the best tasting. Not bitter, nice and smooth.

Review-3 : 
 I bought this coffee on a whim as it was a little cheaper than green mountain.  I really like the flavor, it is a great medium flavor coffee.  I will purchase again.



In [37]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 Do not know a lot about this type of tea, so couldn't compare it to other brands. I can say the product is rather strong tasting, but not unpleasantly so. Since a portion is pretty small, that is not much of a problem anyway. I purchased it after reading about its many health benefits. It does open my eyes in the morning, so the caffeine in it is really nice. As recommended by others, I keep it in the refrigerator inside the tin, in its original package, and it's kept fresh-tasting for a couple of weeks now. The color is very bright green, and it does taste healthy--hopefully, it is.

Review-2 : 
 I have to say that this tea is very helpful during those stressful times.  I don't pay attention to the taste, but next thing I know, I'm like, "WOW - I'm so relaxed."  All that tension slowly melts away.  Great to have around during the time of the month.

Review-3 : 
 This English Breakfast tea from Higgins & Burke, is good: it's basic, good tasting black tea (English Breakfast

In [38]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 We really like Feline Pine Cat Litter, it is a very low maintenance type of product and friendly to the environment.

Review-2 : 
 This is great if you work long shifts and have a small dog.  I have a 7 lb bolognese dog and the breeder litter box trains the dogs from the puppy stage on.  We got her home and she just jumped in the litter box and went.  She has used it now for a year with very little accidents.  It's great and I only replace the litter every 7 - 10 days!

Review-3 : 
 One of my two cats LOVES this and gobbles it down.  The other only likes fancy feast unfortunately.  I love the ingredients and the smell.  It looks just like a can of chicken that any human could eat!  The label says it all--feeding your cat weruva can increase your cat's lives from nine to ten.  Thank you weruva and amazon for providing this fabulous cat food at affordable prices.  Puuurrs!



# Hierarchical Clustering with 10 clusters

In [39]:
model = AgglomerativeClustering(n_clusters=10).fit(data.toarray())
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []
cluster6 = []
cluster7 = []
cluster8 = []
cluster9 = []
cluster10 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    elif model.labels_[i] == 4:
        cluster5.append(reviews[i])
    elif model.labels_[i] == 5:
        cluster6.append(reviews[i])
    elif model.labels_[i] == 6:
        cluster7.append(reviews[i])
    elif model.labels_[i] == 7:
        cluster8.append(reviews[i])
    elif model.labels_[i] == 8:
        cluster9.append(reviews[i])       
    else :
        cluster10.append(reviews[i])

In [40]:


# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5)) 
print("\nNo. of reviews in Cluster-6 : ",len(cluster6))
print("\nNo. of reviews in Cluster-7 : ",len(cluster7))
print("\nNo. of reviews in Cluster-8 : ",len(cluster8))
print("\nNo. of reviews in Cluster-9 : ",len(cluster9))
print("\nNo. of reviews in Cluster-10 : ",len(cluster10))



No. of reviews in Cluster-1 :  3089

No. of reviews in Cluster-2 :  262

No. of reviews in Cluster-3 :  596

No. of reviews in Cluster-4 :  146

No. of reviews in Cluster-5 :  104

No. of reviews in Cluster-6 :  65

No. of reviews in Cluster-7 :  358

No. of reviews in Cluster-8 :  257

No. of reviews in Cluster-9 :  81

No. of reviews in Cluster-10 :  42


READING REVIEWS MANUALLY

In [41]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-2 : 
 Kitchens of India is the closest to a homemade curry we have ever had. We love it. The sauce is great and it is fast, so curry a few times a week is possible. We often make a curry from scratch but it is time consuming and now we can have a good curry whenever we want.  We have also tried the Rogan Josh as well as the Korma. Add some onions and raisins, or a bit of honey. The ingredients have little or no preservatives. The Best!!

Review-3 : 
 This product is light, not too sweet and very satisfying.  Just what it says it is and shows on the box.  There 

In [42]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 I have a serious addiction to this gum but cannot find it anywhere except online. I go through at least a pack a day. I used to chew Peppermint or Spearmint or Winterfrost but I can't chew anything but Exotic Mint any more. The flavor is just right and it lasts forever.<br /><br />Seriously, if you see this gum in a store, buy all of it. It's like little candy gold bullion. I have opted to pay twice as much online for this gum versus gum I can buy on the store shelves just so I can get this particular flavor. Who would do that if the gum weren't the best gum ever? Nobody. That's who.<br /><br />I also wrote Wrigley to see if they'll send me a year-supply for free. Cross your fingers and wish me luck.

Review-2 : 
 We've had this for about a week now. I have a 1 year old Beagle that destroys ever furry "super strong" soft toy, but he isn't interested in a lot of hard toys like tires, etc. We got this hoping that his food motivation would keep him occupied with a toy he can'

In [43]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enjoy.

Review-3 : 
 For all you celiacs out there, this is the best cereal ever.  You can eat with milk in the am or dry in the afternoon as a snack.  Great price point when you order without shipping charge.



In [44]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 I started eating almonds regularly after going on the South Beach Diet. I find them quite delicious . . . so delicious that I have to remind myself that I'm only supposed to eat 15 or so a day.  Oh, well.<br /><br />I could easily eat a handful . . . and still wouldn't feel like I had gotten enough flavor.<br /><br />I had no idea what a cocoa roast almond would taste like, but I was pleasantly surprised that the taste was superb. It reminded me a little of eating a high quality fudge with lots of nuts in it.<br /><br />But the nice surprise was that the cocoa taste lingered in my mouth so that I continued to enjoy the flavor long after the almonds were eaten. Even though the cocoa roasting adds some calories and some fat, I would be satisfied after eating far fewer of these almonds. I plan to buy and eat these on a regular basis.<br /><br />Try them. I think you'll like them!

Review-2 : 
 I absolutely love Justin's Nut Butter Maple Almond!  It is so good I eat it out of 

In [45]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 We really like Feline Pine Cat Litter, it is a very low maintenance type of product and friendly to the environment.

Review-2 : 
 This is great if you work long shifts and have a small dog.  I have a 7 lb bolognese dog and the breeder litter box trains the dogs from the puppy stage on.  We got her home and she just jumped in the litter box and went.  She has used it now for a year with very little accidents.  It's great and I only replace the litter every 7 - 10 days!

Review-3 : 
 One of my two cats LOVES this and gobbles it down.  The other only likes fancy feast unfortunately.  I love the ingredients and the smell.  It looks just like a can of chicken that any human could eat!  The label says it all--feeding your cat weruva can increase your cat's lives from nine to ten.  Thank you weruva and amazon for providing this fabulous cat food at affordable prices.  Puuurrs!



In [46]:
# Three Reviews of cluster 
count=1
for i in range(3):
    if i < len(cluster6):
        print('Review-%d : \n %s\n'%(count,cluster6[i]))
        count +=1

Review-1 : 
 For someone who is vegan and gluten-free, finding snacks out and about can be a challenge. Lucy's cookies are the perfect answer! Not many places carry them, but, luckily, they can be ordered by the case from Amazon for lunchbox and purse stocking!<br /><br />All the flavors are good. Sometimes cookies are broken, but that is to be expected when something like this is shipped, and that just makes the package last longer!

Review-2 : 
 The Back to Nature Fudge Mint cookies and these Fudge Striped cookies are my favorites.<br /><br />The Fudge Striped cookies have a light crunch and a great chocolaty taste.  For some reason when I eat them upside-down they taste even better.  These are a bit healthier than Keebler's which have trans fat in their fudge striped cookies.

Review-3 : 
 I really enjoyed these cookies. They were soft and yummy. Almost like hommade. I would repurchase them again. Quaker is a brand I love and trust for quality.



In [47]:
# Three Reviews of cluster 7
count=1
for i in range(3):
    if i < len(cluster7):
        print('Review-%d : \n %s\n'%(count,cluster7[i]))
        count +=1

Review-1 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-2 : 
 Pilao is my favorite coffe. I have tried several brands from around the globe but so far this is the best tasting. Not bitter, nice and smooth.

Review-3 : 
 I bought this coffee on a whim as it was a little cheaper than green mountain.  I really like the flavor, it is a great medium flavor coffee.  I will purchase again.



In [48]:
# Three Reviews of cluster 8
count=1
for i in range(3):
    if i < len(cluster8):
        print('Review-%d : \n %s\n'%(count,cluster8[i]))
        count +=1

Review-1 : 
 Do not know a lot about this type of tea, so couldn't compare it to other brands. I can say the product is rather strong tasting, but not unpleasantly so. Since a portion is pretty small, that is not much of a problem anyway. I purchased it after reading about its many health benefits. It does open my eyes in the morning, so the caffeine in it is really nice. As recommended by others, I keep it in the refrigerator inside the tin, in its original package, and it's kept fresh-tasting for a couple of weeks now. The color is very bright green, and it does taste healthy--hopefully, it is.

Review-2 : 
 I have to say that this tea is very helpful during those stressful times.  I don't pay attention to the taste, but next thing I know, I'm like, "WOW - I'm so relaxed."  All that tension slowly melts away.  Great to have around during the time of the month.

Review-3 : 
 This English Breakfast tea from Higgins & Burke, is good: it's basic, good tasting black tea (English Breakfast

In [49]:
# Three Reviews of cluster 9
count=1
for i in range(3):
    if i < len(cluster9):
        print('Review-%d : \n %s\n'%(count,cluster9[i]))
        count +=1

Review-1 : 
 I am helping a 71 yr.old sufferer from Parkinson's to completely recover from the syndrome, partly by replacing animal-product fish oil with pure virgin coconut oil, and with very satisfying results!

Review-2 : 
 After doing research for some time on  coconut oils, I settled on Tropical Traditions. I've been using it for years and usually put some in my oatmeal. I've also used it on my skin after extended sun exposure. I've been very happy with the product. In my research I found that Coconut Oil has very strong anti-viral properties. A friend of mine's daughter came down with a bad case of Mono. She was told she would be out of Softball for several months. I gave him a jar and instructed him to start her out with one spoonful in the a.m. and then work up to 3 a day. Within several days she was showing signs of being better and within two weeks was back in the Texas Summer heat playing softball and never had any issues. Was it the Coconut Oil? I think yes but regardless, 

In [50]:
# Three Reviews of cluster 10
count=1
for i in range(3):
    if i < len(cluster10):
        print('Review-%d : \n %s\n'%(count,cluster10[i]))
        count +=1

Review-1 : 
 Bought the popper about two years ago and have been enjoying the delicious fresh buttery salty (as I want) best popcorn ever.  love it and it's a staple snack in our house. Would never micorwave corn again.

Review-2 : 
 I have eaten all kinds of popcorn for many years.  Many years ago, we made popcorn the old fashioned way; in a pot on the stove.  I remember it being crisp and crunchy.  Then, along came microwave popcorn.  I tried many different brands and none of them had the same crisp, and sometimes they were just plain mushy. I thought this was just a common attribute among microwavable popcorn products until several years ago when I tried making it the old fashioned way with the same results.<br /><br />What I have come to realize is, it's not how you prepare the popcorn, it's whether or not it's "organic."  The first time I tried "Newman's Own," it had that crisp and crunchy texture that I remembered from so many years ago.  Perfect!

Review-3 : 
 Pricey and hard to

# Word2Vec

In [51]:
sent_x = []
for sent in x :
    sent_x.append(sent.split())
  
    
# Train your own Word2Vec model using your own train text corpus 
# min_count = 5 considers only words that occured atleast 5 times
w2v_model=Word2Vec(sent_x,min_count=5,size=50, workers=4)

w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))

number of words that occured minimum 5 times  3177


In [52]:
train_vectors = []; 
for sent in sent_x:
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent: # 
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    train_vectors.append(sent_vec)
    
data = train_vectors

# Hierarchical Clustering with 2 clusters

In [53]:
model = AgglomerativeClustering(n_clusters=2).fit(data)

reviews = my_final['Text'].values
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    else :
        cluster2.append(reviews[i])
 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))

No. of reviews in Cluster-1 :  2814

No. of reviews in Cluster-2 :  2186


In [54]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    print('Review-%d : \n %s\n'%(count,cluster1[i]))
    count +=1

Review-1 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enjoy.

Review-2 : 
 Kitchens of India is the closest to a homemade curry we have ever had. We love it. The sauce is great and it is fast, so curry a few times a week is possible. We often make a curry from scratch but it is time consuming and now we can have a good curry whenever we want.  We have also tried the Rogan Josh as well as the Korma. Add some onions and raisins, or a bit of honey. The ingredients have little or no preservatives. The Best!!

Review-3 : 
 Teff in general is a delicious healthy grain, and this Teff is no different.<br />Cook according to pkg. directions and add to morning oatmeal for a hearty healthy breakfast.<br />If I had one wish it would be for it to be "Organic Brown Teff."



In [55]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-3 : 
 For all you celiacs out there, this is the best cereal ever.  You can eat with milk in the am or dry in the afternoon as a snack.  Great price point when you order without shipping cha

# Hierarchical Clustering with 5 clusters

In [56]:
model = AgglomerativeClustering(n_clusters=5).fit(data)

# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    else :
        cluster5.append(reviews[i]) 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5))

No. of reviews in Cluster-1 :  2221

No. of reviews in Cluster-2 :  751

No. of reviews in Cluster-3 :  593

No. of reviews in Cluster-4 :  473

No. of reviews in Cluster-5 :  962


In [ ]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

In [57]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-2 : 
 For all you celiacs out there, this is the best cereal ever.  You can eat with milk in the am or dry in the afternoon as a snack.  Great price point when you order without shipping charge.

Review-3 : 
 These arrived quickly but the box was really banged up. had to open a couple of bags just to be sure the chips weren't too broken up since I'm selling these in a vending machine, they were pretty crunched, I can't sell these.



In [58]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-2 : 
 I Have been ordering this k cup cappucino drink for awhile now and have never had a problem! Love it tastes good!

Review-3 : 
 This water taste great. We love the bottle. Perfect water for everyone :) I love that it is 1.5 liters because I am trying to drink more water. You will love this water.



In [59]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 Had this in cart and ready to buy when after a bit of research (info not readily available) realized the excessive cost of shipping.  Such cost make this product less than feasible to try?

Review-2 : 
 I love this product, well, my cat loves this product. They no longer sell it in the pet stores, I was so grateful to find it on Amazon, my true shopping place.<br /><br />The service was excellent, fast and on time. Would buy again

Review-3 : 
 I do not live in the United States, and I am used of Splenda, and where I live nowadays, splenda is very expensive and not easy to find, and I found out that if I order it online from Amazon.com, I will get it much cheaper than where I live.



In [60]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This product is light, not too sweet and very satisfying.  Just what it says it is and shows on the box.  There is no sacrifice of delectable quality in the pursuit of a gluten free product at all.  Needs no frosting at all! Guests told me how wonderful the cake was even when they didn't know it was good for everybody, including us gluten free folk.

Review-3 : 
 My 8-month baby always screws up his face like he's being poisoned when you give him something new, so I thought he'd need some time to get used to this.  Not true!  He loved this from the first bite.  And while he tends not to like foods with 

# Hierarchical Clustering with 10 clusters

In [61]:
model = AgglomerativeClustering(n_clusters=10).fit(data)
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []
cluster6 = []
cluster7 = []
cluster8 = []
cluster9 = []
cluster10 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    elif model.labels_[i] == 4:
        cluster5.append(reviews[i])
    elif model.labels_[i] == 5:
        cluster6.append(reviews[i])
    elif model.labels_[i] == 6:
        cluster7.append(reviews[i])
    elif model.labels_[i] == 7:
        cluster8.append(reviews[i])
    elif model.labels_[i] == 8:
        cluster9.append(reviews[i])       
    else :
        cluster10.append(reviews[i])

In [62]:
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5)) 
print("\nNo. of reviews in Cluster-6 : ",len(cluster6))
print("\nNo. of reviews in Cluster-7 : ",len(cluster7))
print("\nNo. of reviews in Cluster-8 : ",len(cluster8))
print("\nNo. of reviews in Cluster-9 : ",len(cluster9))
print("\nNo. of reviews in Cluster-10 : ",len(cluster10))

No. of reviews in Cluster-1 :  593

No. of reviews in Cluster-2 :  962

No. of reviews in Cluster-3 :  893

No. of reviews in Cluster-4 :  732

No. of reviews in Cluster-5 :  447

No. of reviews in Cluster-6 :  392

No. of reviews in Cluster-7 :  431

No. of reviews in Cluster-8 :  320

No. of reviews in Cluster-9 :  26

No. of reviews in Cluster-10 :  204


In [63]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-2 : 
 I Have been ordering this k cup cappucino drink for awhile now and have never had a problem! Love it tastes good!

Review-3 : 
 This water taste great. We love the bottle. Perfect water for everyone :) I love that it is 1.5 liters because I am trying to drink more water. You will love this water.



In [64]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This product is light, not too sweet and very satisfying.  Just what it says it is and shows on the box.  There is no sacrifice of delectable quality in the pursuit of a gluten free product at all.  Needs no frosting at all! Guests told me how wonderful the cake was even when they didn't know it was good for everybody, including us gluten free folk.

Review-3 : 
 My 8-month baby always screws up his face like he's being poisoned when you give him something new, so I thought he'd need some time to get used to this.  Not true!  He loved this from the first bite.  And while he tends not to like foods with 

In [65]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 Teff in general is a delicious healthy grain, and this Teff is no different.<br />Cook according to pkg. directions and add to morning oatmeal for a hearty healthy breakfast.<br />If I had one wish it would be for it to be "Organic Brown Teff."

Review-2 : 
 I made this cake for Christmas 2008.  My family thought this cake was dry and said that didn't taste good.  It actually left a bad aftertaste in the mouth.  I frosted it with Cherrybrook Kitchen Chocolate Frosting Gluten Free and they loved that.  They did not want me to ever repeat this cake but they said they would be happy to have the frosting again.

Review-3 : 
 We love Sunbutter! I don't have to worry about using it, as it and the company is completely PEANUT FREE!. Sunbutter taste great, unlike Soy Butter which is gritty and awful tasting. I use Sunbutter for anything that I would use peanut butter for cookies, frostings and candies, dips, etc. Sunbutter has the same consistency and texture as peanut butter and 

In [66]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 Kitchens of India is the closest to a homemade curry we have ever had. We love it. The sauce is great and it is fast, so curry a few times a week is possible. We often make a curry from scratch but it is time consuming and now we can have a good curry whenever we want.  We have also tried the Rogan Josh as well as the Korma. Add some onions and raisins, or a bit of honey. The ingredients have little or no preservatives. The Best!!

Review-2 : 
 I was looking for yeast with no added ingredients, and this is not it...<br />Ingredients: Yeast, sorbitan monostearate, ascorbic acid<br />It also came hot off the UPS truck, and I hesitate to open it for fear it is activated and dead.  Will return instead.<br />Not what I was looking for, won't purchase again.

Review-3 : 
 My grown kids always dieting love the flavors and that it is carbonated, especially the Pomegranate Berry, my grandkids, whose parents are always watching what they are drinking, love them just as much.  I had 

In [67]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 Had this in cart and ready to buy when after a bit of research (info not readily available) realized the excessive cost of shipping.  Such cost make this product less than feasible to try?

Review-2 : 
 I love this product, well, my cat loves this product. They no longer sell it in the pet stores, I was so grateful to find it on Amazon, my true shopping place.<br /><br />The service was excellent, fast and on time. Would buy again

Review-3 : 
 I do not live in the United States, and I am used of Splenda, and where I live nowadays, splenda is very expensive and not easy to find, and I found out that if I order it online from Amazon.com, I will get it much cheaper than where I live.



In [68]:
# Three Reviews of cluster 6
count=1
for i in range(3):
    if i < len(cluster6):
        print('Review-%d : \n %s\n'%(count,cluster6[i]))
        count +=1

Review-1 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enjoy.

Review-2 : 
 I have this item to ship monthly.  I think this is the best honey that I have found.  I tried almost all local honeys, I wanted uncook & unfiltered honey for the best health benefit. I found this is by far the best flavor and consistency.  My honey arrives on time and in good condition.

Review-3 : 
 Do not know a lot about this type of tea, so couldn't compare it to other brands. I can say the product is rather strong tasting, but not unpleasantly so. Since a portion is pretty small, that is not much of a problem anyway. I purchased it after reading about its many health benefits. It does open my eyes in the morning, so the caffeine in it is really nice. As recommended by others, I keep it in the refrigerator inside the tin, in its original package, and it's kept fresh-tast

In [69]:
# Three Reviews of cluster 7
count=1
for i in range(3):
    if i < len(cluster7):
        print('Review-%d : \n %s\n'%(count,cluster7[i]))
        count +=1

Review-1 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-2 : 
 For all you celiacs out there, this is the best cereal ever.  You can eat with milk in the am or dry in the afternoon as a snack.  Great price point when you order without shipping charge.

Review-3 : 
 These arrived quickly but the box was really banged up. had to open a couple of bags just to be sure the chips weren't too broken up since I'm selling these in a vending machine, they were pretty crunched, I can't sell these.



In [70]:
# Three Reviews of cluster 8
count=1
for i in range(3):
    if i < len(cluster8):
        print('Review-%d : \n %s\n'%(count,cluster8[i]))
        count +=1

Review-1 : 
 Like others, I cannot order this product w/o an ingredients list.  Amazon, people with food allergies, intolerances, or other dietary restrictions need complete information!  Thank you and please resend the G-F for Betty Crocker notice when ingredients lists are included.

Review-2 : 
 My son is 7 months old and this has always been his favorite baby food - it is nice a thick, which he loves!

Review-3 : 
 The dog food got good ratings at consumers, so I bought it and the dogs just love it.. It seems to be a very healtful and complete food.. I just add a little hot water and its gobbled up in no time



In [71]:
# Three Reviews of cluster 9
count=1
for i in range(3):
    if i < len(cluster9):
        print('Review-%d : \n %s\n'%(count,cluster10[i]))
        count +=1

Review-1 : 
 Smells tempting when its being brewed but the flavor just isnt there.  I will not purchase this again.  I gave the other 2 bags away as gifts (because my roommates weren't impressed with the flavor either).

Review-2 : 
 Great drink for working out. I was paying $5 at 24 hour for each drink now got a case of them for cheap.

Review-3 : 
 Wanted to try the best coffee in the world. Recieved as promised and enjoyed from the first cup. Would order again.



In [72]:
# Three Reviews of cluster 10
count=1
for i in range(3):
    if i < len(cluster10):
        print('Review-%d : \n %s\n'%(count,cluster10[i]))
        count +=1

Review-1 : 
 Smells tempting when its being brewed but the flavor just isnt there.  I will not purchase this again.  I gave the other 2 bags away as gifts (because my roommates weren't impressed with the flavor either).

Review-2 : 
 Great drink for working out. I was paying $5 at 24 hour for each drink now got a case of them for cheap.

Review-3 : 
 Wanted to try the best coffee in the world. Recieved as promised and enjoyed from the first cup. Would order again.



# TFIDF-Word2Vec

In [73]:
tf_idf_vect = TfidfVectorizer()

# final_tf_idf1 is the sparse matrix with row= sentence, col=word and cell_val = tfidf
final_tf_idf1 = tf_idf_vect.fit_transform(x)

# tfidf words/col-names
tfidf_feat = tf_idf_vect.get_feature_names()

# compute TFIDF Weighted Word2Vec for each review for sent_x .
tfidf_vectors = []; 
row=0;
for sent in sent_x: 
    sent_vec = np.zeros(50) 
    weight_sum =0; 
    for word in sent: 
        if word in w2v_words:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tf_idf = final_tf_idf1[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_vectors.append(sent_vec)
    row += 1 
    
data = tfidf_vectors

# Hierarchical Clustering with 2 clusters

In [74]:
model = AgglomerativeClustering(n_clusters=2).fit(data)

reviews = my_final['Text'].values
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    else :
        cluster2.append(reviews[i])
 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))

No. of reviews in Cluster-1 :  3830

No. of reviews in Cluster-2 :  1170


In [75]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-3 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enj

In [76]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 Smells tempting when its being brewed but the flavor just isnt there.  I will not purchase this again.  I gave the other 2 bags away as gifts (because my roommates weren't impressed with the flavor either).

Review-2 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-3 : 
 I Have been ordering this k cup cappucino drink for awhile now and have never had a problem! Love it tastes good!



# Hierarchical Clustering with 5 clusters

In [77]:
model = AgglomerativeClustering(n_clusters=5).fit(data)

# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    else :
        cluster5.append(reviews[i]) 
        
# Number of reviews in different clusters
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5))

No. of reviews in Cluster-1 :  1170

No. of reviews in Cluster-2 :  812

No. of reviews in Cluster-3 :  2173

No. of reviews in Cluster-4 :  490

No. of reviews in Cluster-5 :  355


In [78]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 Smells tempting when its being brewed but the flavor just isnt there.  I will not purchase this again.  I gave the other 2 bags away as gifts (because my roommates weren't impressed with the flavor either).

Review-2 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-3 : 
 I Have been ordering this k cup cappucino drink for awhile now and have never had a problem! Love it tastes good!



In [79]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 These are just great.  Even my dog liked them, stealing a pack off a coffee table and eating darned near the whole thing.

Review-3 : 
 I was looking for yeast with no added ingredients, and this is not it...<br />Ingredients: Yeast, sorbitan monostearate, ascorbic acid<br />It also came hot off the UPS truck, and I hesitate to open it for fear it is activated and dead.  Will return instead.<br />Not what I was looking for, won't purchase again.



In [80]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-2 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enjoy.

Review-3 : 
 For all you celiacs out there, this is the best cereal ever.  You can eat with milk in the am or dry in the afternoon as a snack.  Great price point when you order without shipping charge.



In [81]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 These arrived quickly but the box was really banged up. had to open a couple of bags just to be sure the chips weren't too broken up since I'm selling these in a vending machine, they were pretty crunched, I can't sell these.

Review-2 : 
 I have a serious addiction to this gum but cannot find it anywhere except online. I go through at least a pack a day. I used to chew Peppermint or Spearmint or Winterfrost but I can't chew anything but Exotic Mint any more. The flavor is just right and it lasts forever.<br /><br />Seriously, if you see this gum in a store, buy all of it. It's like little candy gold bullion. I have opted to pay twice as much online for this gum versus gum I can buy on the store shelves just so I can get this particular flavor. Who would do that if the gum weren't the best gum ever? Nobody. That's who.<br /><br />I also wrote Wrigley to see if they'll send me a year-supply for free. Cross your fingers and wish me luck.

Review-3 : 
 These cashews are of de

In [82]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 My 8-month baby always screws up his face like he's being poisoned when you give him something new, so I thought he'd need some time to get used to this.  Not true!  He loved this from the first bite.  And while he tends not to like foods with a lot of texture yet, the rice-in-sauce texture of this food doesn't bother him at all.

Review-2 : 
 My son is 7 months old and this has always been his favorite baby food - it is nice a thick, which he loves!

Review-3 : 
 The dog food got good ratings at consumers, so I bought it and the dogs just love it.. It seems to be a very healtful and complete food.. I just add a little hot water and its gobbled up in no time



# Hierarchical Clustering with 10 clusters

In [83]:
model = AgglomerativeClustering(n_clusters=10).fit(data)
# Getting all the reviews in different clusters
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
cluster5 = []
cluster6 = []
cluster7 = []
cluster8 = []
cluster9 = []
cluster10 = []

for i in range(model.labels_.shape[0]):
    if model.labels_[i] == 0:
        cluster1.append(reviews[i])
    elif model.labels_[i] == 1:
        cluster2.append(reviews[i])
    elif model.labels_[i] == 2:
        cluster3.append(reviews[i])
    elif model.labels_[i] == 3:
        cluster4.append(reviews[i])
    elif model.labels_[i] == 4:
        cluster5.append(reviews[i])
    elif model.labels_[i] == 5:
        cluster6.append(reviews[i])
    elif model.labels_[i] == 6:
        cluster7.append(reviews[i])
    elif model.labels_[i] == 7:
        cluster8.append(reviews[i])
    elif model.labels_[i] == 8:
        cluster9.append(reviews[i])       
    else :
        cluster10.append(reviews[i])

In [84]:
print("No. of reviews in Cluster-1 : ",len(cluster1))
print("\nNo. of reviews in Cluster-2 : ",len(cluster2))
print("\nNo. of reviews in Cluster-3 : ",len(cluster3))
print("\nNo. of reviews in Cluster-4 : ",len(cluster4))
print("\nNo. of reviews in Cluster-5 : ",len(cluster5)) 
print("\nNo. of reviews in Cluster-6 : ",len(cluster6))
print("\nNo. of reviews in Cluster-7 : ",len(cluster7))
print("\nNo. of reviews in Cluster-8 : ",len(cluster8))
print("\nNo. of reviews in Cluster-9 : ",len(cluster9))
print("\nNo. of reviews in Cluster-10 : ",len(cluster10))

No. of reviews in Cluster-1 :  330

No. of reviews in Cluster-2 :  1409

No. of reviews in Cluster-3 :  649

No. of reviews in Cluster-4 :  446

No. of reviews in Cluster-5 :  355

No. of reviews in Cluster-6 :  419

No. of reviews in Cluster-7 :  764

No. of reviews in Cluster-8 :  366

No. of reviews in Cluster-9 :  71

No. of reviews in Cluster-10 :  191


In [85]:
# Three Reviews of cluster 1
count=1
for i in range(3):
    if i < len(cluster1):
        print('Review-%d : \n %s\n'%(count,cluster1[i]))
        count +=1

Review-1 : 
 This is a good deal.  Pack of 2, one for me and one to give as a gift.  Very easy to use.  I use it at night to make a cup of decaf.  Makes good coffee and you can make coffee your desired strength.  I use 1 scoop of coffee for every 5oz of water.

Review-2 : 
 This water taste great. We love the bottle. Perfect water for everyone :) I love that it is 1.5 liters because I am trying to drink more water. You will love this water.

Review-3 : 
 This English Breakfast tea from Higgins & Burke, is good: it's basic, good tasting black tea (English Breakfast blend) and is tasty.  I wasn't blown away by the flavor, but it was good: I enjoyed the tea, it brewed up quickly in these nice tea bags (I recommend about one minute of brew time), and then added a splash of cream: it was quite nice!



In [86]:
# Three Reviews of cluster 2
count=1
for i in range(3):
    if i < len(cluster2):
        print('Review-%d : \n %s\n'%(count,cluster2[i]))
        count +=1

Review-1 : 
 Kitchens of India is the closest to a homemade curry we have ever had. We love it. The sauce is great and it is fast, so curry a few times a week is possible. We often make a curry from scratch but it is time consuming and now we can have a good curry whenever we want.  We have also tried the Rogan Josh as well as the Korma. Add some onions and raisins, or a bit of honey. The ingredients have little or no preservatives. The Best!!

Review-2 : 
 Teff in general is a delicious healthy grain, and this Teff is no different.<br />Cook according to pkg. directions and add to morning oatmeal for a hearty healthy breakfast.<br />If I had one wish it would be for it to be "Organic Brown Teff."

Review-3 : 
 I have this item to ship monthly.  I think this is the best honey that I have found.  I tried almost all local honeys, I wanted uncook & unfiltered honey for the best health benefit. I found this is by far the best flavor and consistency.  My honey arrives on time and in good co

In [87]:
# Three Reviews of cluster 3
count=1
for i in range(3):
    if i < len(cluster3):
        print('Review-%d : \n %s\n'%(count,cluster3[i]))
        count +=1

Review-1 : 
 Smells tempting when its being brewed but the flavor just isnt there.  I will not purchase this again.  I gave the other 2 bags away as gifts (because my roommates weren't impressed with the flavor either).

Review-2 : 
 I Have been ordering this k cup cappucino drink for awhile now and have never had a problem! Love it tastes good!

Review-3 : 
 Preparation could not be easier.  The flavor was wonderful.  My son who almost never salts anything found it a bit salty for him.  My Mom who should not eat salt said she thought it was a bit salty too.



In [88]:
# Three Reviews of cluster 4
count=1
for i in range(3):
    if i < len(cluster4):
        print('Review-%d : \n %s\n'%(count,cluster4[i]))
        count +=1

Review-1 : 
 I like the mint, but the only reason I have bought these mint are because I like the tin. The tins are very nice. There are a lot of project I can build using these candy tins. The mint is not bad either, very freshing.<br /><br />They sell these mint for $2.50/tin here in San Francisco, so 12 tins will cost me $30, and I saved about 10 dollars to get it here.

Review-2 : 
 These are always going to be more expensive than buying the single big pack - remember when oreos were packed into three plastic tubes in a plastic box.<br /><br />These oreos always come out crunchy with a very pliable filling that can be easily nudged-rolled off if you don't like the filling.  If Nabisco sold individually wrapped cookies, I'd pay the extra money as I usually only want one cookie.<br /><br />I will always choose to buy oreos packaged in as small a serving as possible despite the expense unless I get some kind of vacuum sealer.

Review-3 : 
 When I purchaed this item, I thought it will 

In [89]:
# Three Reviews of cluster 5
count=1
for i in range(3):
    if i < len(cluster5):
        print('Review-%d : \n %s\n'%(count,cluster5[i]))
        count +=1

Review-1 : 
 My 8-month baby always screws up his face like he's being poisoned when you give him something new, so I thought he'd need some time to get used to this.  Not true!  He loved this from the first bite.  And while he tends not to like foods with a lot of texture yet, the rice-in-sauce texture of this food doesn't bother him at all.

Review-2 : 
 My son is 7 months old and this has always been his favorite baby food - it is nice a thick, which he loves!

Review-3 : 
 The dog food got good ratings at consumers, so I bought it and the dogs just love it.. It seems to be a very healtful and complete food.. I just add a little hot water and its gobbled up in no time



In [90]:
# Three Reviews of cluster 6
count=1
for i in range(3):
    if i < len(cluster6):
        print('Review-%d : \n %s\n'%(count,cluster6[i]))
        count +=1

Review-1 : 
 These arrived quickly but the box was really banged up. had to open a couple of bags just to be sure the chips weren't too broken up since I'm selling these in a vending machine, they were pretty crunched, I can't sell these.

Review-2 : 
 I have a serious addiction to this gum but cannot find it anywhere except online. I go through at least a pack a day. I used to chew Peppermint or Spearmint or Winterfrost but I can't chew anything but Exotic Mint any more. The flavor is just right and it lasts forever.<br /><br />Seriously, if you see this gum in a store, buy all of it. It's like little candy gold bullion. I have opted to pay twice as much online for this gum versus gum I can buy on the store shelves just so I can get this particular flavor. Who would do that if the gum weren't the best gum ever? Nobody. That's who.<br /><br />I also wrote Wrigley to see if they'll send me a year-supply for free. Cross your fingers and wish me luck.

Review-3 : 
 These cashews are of de

In [91]:
# Three Reviews of cluster 7
count=1
for i in range(3):
    if i < len(cluster7):
        print('Review-%d : \n %s\n'%(count,cluster7[i]))
        count +=1

Review-1 : 
 This is my second review request for this mustard.  Again, I will say that it is my favorite mustard and hard to find in select stores.  I was so happy to see that I could purchase it on Amazon.  This is sure a great place to shop from my mustard to my hard to find shampoos.  This mustard is wonderful on all kind of meats.  My husband is not a mustard lover like me, but he does like this brand and selection.

Review-2 : 
 Great product. Great price. I use it as a substitute for white sugar in everything from my coffee to cooking and baking. I use Amazon's Subscribe and Save program for this product. Enjoy.

Review-3 : 
 For all you celiacs out there, this is the best cereal ever.  You can eat with milk in the am or dry in the afternoon as a snack.  Great price point when you order without shipping charge.



In [92]:
# Three Reviews of cluster 8
count=1
for i in range(3):
    if i < len(cluster8):
        print('Review-%d : \n %s\n'%(count,cluster8[i]))
        count +=1

Review-1 : 
 These are just great.  Even my dog liked them, stealing a pack off a coffee table and eating darned near the whole thing.

Review-2 : 
 I was looking for yeast with no added ingredients, and this is not it...<br />Ingredients: Yeast, sorbitan monostearate, ascorbic acid<br />It also came hot off the UPS truck, and I hesitate to open it for fear it is activated and dead.  Will return instead.<br />Not what I was looking for, won't purchase again.

Review-3 : 
 We love Sunbutter! I don't have to worry about using it, as it and the company is completely PEANUT FREE!. Sunbutter taste great, unlike Soy Butter which is gritty and awful tasting. I use Sunbutter for anything that I would use peanut butter for cookies, frostings and candies, dips, etc. Sunbutter has the same consistency and texture as peanut butter and doesn't need refrigeration. When using Sunbutter for baking you do have to cut back to about 1/2 of the baking soda and/or baking powder or you can add a little bit 

In [93]:
# Three Reviews of cluster 9
count=1
for i in range(3):
    if i < len(cluster9):
        print('Review-%d : \n %s\n'%(count,cluster9[i]))
        count +=1

Review-1 : 
 My daughter eats these bars on a daily basis.  I usually purchase these at the local grocery store and thought is would be cost effective to order through Amazon.  My grocery store charges $.99 per bar.  Amazon's price is $1.44 per bar.

Review-2 : 
 These sell in organic shops and deli's for twice what you will pay on Amazon.  They are always a hit at parties or family get togethers so you will need a case which is what Amazon sells.

Review-3 : 
 I ordered the 10-bag product and only received 6 bags.  Do not order from this company as it does not fulfill it's obligations to their customers.  I could have tolerated the 10-day wait for the order to come in if it had been exactly what I ordered, but it wasn't.



In [94]:
# Three Reviews of cluster 10
count=1
for i in range(3):
    if i < len(cluster10):
        print('Review-%d : \n %s\n'%(count,cluster10[i]))
        count +=1

Review-1 : 
 Very strong Ginger flavor, mild carbonation, I like that. You must like Ginger Ale to like this. The small bottles are about the right size, as their is more flavor in 7 Ounce bottle then anyone else has in 12.

Review-2 : 
 Even the "plain" flavor was much better than I expected. I had my heart set on getting the Vanilla Bean, but it was often out of stock. I opted for to try other flavors to test others and was surprised by how good tasting they all were. I actually enjoy matching my workout moods with the different flavors! Now I will just get this mixed flavors box every time.

Review-3 : 
 This is definitely my favorite new drink mix in the Mocktail line by Crystal Light.  It's a great tasting lime flavored drink.  The artificial sweetener gives this a very slight aftertaste, but just slight.  And I like the fact that it is only 5 calories per serving.  Each pouch makes enough to fill a large pitcher.  There isn't any type of alcohol flavor, which is fine with me.  It